## **Async Methods with AnthropicModel**

## **Introduction:**

Anthropic offers a suite of advanced **Large Language Models (LLMs)** optimized for safety, reliability, and contextual understanding, making them ideal for a wide range of conversational and text generation tasks. Models like *claude-3-haiku-20240307* and *Claude-2* are designed to handle complex NLP applications with efficiency and precision. This notebook focuses on using Anthropic's LLMs in **asynchronous** workflows, enabling concurrent processing and non-blocking execution for faster and more efficient task management.  

**This notebook will:**

- Introduce setup and initialization for Anthropic models within **Swarmauri**.  
- Demonstrate **asynchronous** text generation, streaming responses, and contextual query handling.  
- Provide examples showcasing Anthropic's model capabilities for managing concurrent conversations and batch processing.  

By the end of this notebook, you will have a thorough understanding of how to leverage Anthropic's LLMs to build scalable and efficient NLP applications with asynchronous techniques. 

---

### **One More Thing**

Before diving into the implementation of asynchronous methods, let’s take a moment to understand asynchronous processing.

---

### **What is Asynchronous Processing?**

Asynchronous processing allows tasks to run independently of the main program flow, enabling multiple operations to execute concurrently(at the same time) without blocking or waiting for one task to complete before starting another. 

---

### **Why Use Asynchronous Methods?**  

1. **Efficiency**: It improves performance when handling multiple requests, as it doesn't wait for one task to complete before starting the next.  
2. **Scalability**: Ideal for real-time applications where fast response times are crucial.  
3. **Non-Blocking I/O**: Particularly useful when working with APIs, databases, or other external services.  

---

### **When Should You Use Asynchronous Methods?**  

- When building applications that need to handle multiple users simultaneously.  
- When working on tasks that rely on external systems like APIs or databases, which may introduce latency.  
- For real-time use cases like chat applications, where a fast and smooth user experience is essential.  

---

By now, you should know what asynchronous programming is and when you should use it. 
If you are still confused about it, a simple google search will surely give you more insights.

**Let's now go into the practical side of things**

# **Setup and Configuration**

## Import the necessary classes

In [2]:
from swarmauri.llms.concrete.AnthropicModel import AnthropicModel  # This is the actual AnthropicModel class
from swarmauri.conversations.concrete.Conversation import Conversation  # For handling conversation logic and flow
from swarmauri.messages.concrete.HumanMessage import HumanMessage  # For adding user input or human messages
from swarmauri.messages.concrete.SystemMessage import SystemMessage  # For adding system-level instructions (System Prompts)

### Load your API KEY from your environment variables
- Make sure you have python-dotenv installed if not, run `pip install python-dotenv` so you can install it.
- To get your API KEY, click [HERE](https://console.anthropic.com/settings/keys)

In [23]:
import os
from dotenv import load_dotenv
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")  

### Initialize Anthropic Model
- Note: You can as well input your api key directly, but it's better to load from env file
- Also, the `name` arguments is an optional argument that allows you to input a model from the list of allowed_models. 

In [24]:
model = AnthropicModel(api_key=ANTHROPIC_API_KEY, name="claude-3-haiku-20240307")

In [29]:
conversation = Conversation()
conversation.add_message(HumanMessage(content="Who is the founder of Python Programming Language?."))

In [30]:
# Use the asynchronous method `apredict` to generate a response
result = await model.apredict(conversation=conversation)

# Retrieve and display the content of the prediction
answer = result.get_last().content
print("Answer:", answer)

Answer: The Python programming language was created by Guido van Rossum.

Guido van Rossum is a Dutch programmer who is best known as the creator and "Benevolent Dictator for Life" of the Python programming language. He started working on Python in the late 1980s at the National Research Institute for Mathematics and Computer Science (CWI) in the Netherlands.

The first version of Python was released in 1991, and since then, Guido van Rossum has been the leading figure in the development and evolution of the language. He has guided the language's growth and direction, and his vision and leadership have been instrumental in making Python one of the most popular and widely-used programming languages today.

Guido van Rossum's contributions to the development of Python, as well as his involvement in the Python community, have earned him recognition and respect in the programming world. He is widely regarded as the founder and principal architect of the Python programming language.


### **2. Adding System Context to Your Conversation**

System context allows you to guide the model's behavior by providing it with background instructions or a specific role to follow. By defining the system context, you can specify how the model should respond, the tone it should adopt, and the scope of its expertise. This ensures the model generates accurate and relevant responses tailored to your needs.

For example, you can define the system context to:

- Act as a Python programming tutor or expert.  
- Provide step-by-step explanations for technical problems.  
- Focus on a particular domain, such as data science or web development.  

**Below, we will define a system context for a Python programming tutor specializing in beginner to advanced concepts.**

In [7]:
python_tutor_system_context = """
You are a Python programming tutor with expertise ranging from beginner to advanced topics. 
Your role is to guide learners with clear, concise, and step-by-step explanations, 
helping them understand Python concepts, write efficient code, and solve problems. 
Be patient and provide practical examples when explaining concepts.
"""

In [15]:
conversation.add_message(SystemMessage(content=python_tutor_system_context))
conversation.add_message(HumanMessage(content="Can you explain the difference between a list and a tuple in Python?"))

In [16]:
# Use the asynchronous method `apredict` to generate a response
result = await model.apredict(conversation=conversation)

# Retrieve and display the content of the prediction
answer = result.get_last().content
print("Answer:", answer)

Answer: Certainly! The main differences between lists and tuples in Python are:

1. Mutability:
   - Lists are mutable, meaning you can add, remove, or modify elements in a list after it has been created.
   - Tuples are immutable, meaning you cannot modify the elements in a tuple after it has been created.

2. Syntax:
   - Lists are defined using square brackets `[]`, e.g., `my_list = [1, 2, 3]`.
   - Tuples are defined using parentheses `()`, e.g., `my_tuple = (1, 2, 3)`.

3. Usage:
   - Lists are commonly used when you need to store a collection of items that may need to be modified.
   - Tuples are often used to represent a fixed set of values, such as coordinates, database records, or configuration settings.

4. Performance:
   - Tuples are generally faster than lists because they are immutable, and the interpreter can optimize their storage and access.

5. Hashability:
   - Tuples are hashable, meaning they can be used as


---
### **3. Advanced Asynchronous Processing Techniques**

#### 1. **Asynchronous Streaming:**  
Using the `astream` method, responses are generated and displayed **token by token** asynchronously as they are received. This is ideal for real-time applications where partial answers are required progressively without blocking other tasks in the process.

For example, when using ChatGPT, you may notice that answers **appear gradually, word by word**, as the model processes and sends the response in real time. This behavior can be efficiently replicated using the `astream` method, allowing concurrent tasks to continue running while the response is being generated.

In [17]:
human_message = HumanMessage(content="In what areas of technology does python thrive?")
conversation.add_message(human_message)

collected_tokens = []
async for token in model.astream(conversation=conversation):
    print(token) 

Python is
 a versatile programming language that has found applications in a
 wide range of technology areas. Some of the key areas
 where Python thrives include:

1. Data Science and Machine Learning
:
   - Python is a popular language for data analysis
, statistical computing, and building machine learning models.
 Libraries like NumPy, Pandas,
 Matplotlib, Scikit-learn, and T
ensorFlow make Python a go-to choice
 for data scientists and machine learning practitioners.

2.
 Web Development:
   - Python's web
 frameworks, such as Django, Flask, an
d Pyramid, are widely used for building robust and scalable web
 applications.

3. Automation and Scri
pting:
   - Python's simplicity an
d readability make it an excellent choice for autom
ating various tasks, from system administration to file management
 and web scraping.

4. Scientific
 and Numerical Computing:
   - Python's scientific
 computing libraries, including NumPy, S
ciPy, and Matplotlib, are widely used in
 fields like physics,

As you can see, the answer did not appear all at once when you ran the code. Instead, it appeared in parts, just as it would when using the Anthropic web interface.  

This is the beauty of the `astream` method, as it allows users to see the answers as they are generated, keeping them engaged throughout the process.

#### **2. Abatch Processing Multiple Conversations**  
Imagine you need to send **multiple questions** at once or handle queries from **several users simultaneously**. The `abatch` method allows you to process these requests **asynchronously** in a single call, enabling faster and more efficient handling of multiple conversations without blocking execution.

In [19]:
conversations = []
for prompt in ["If I want to learn python, how do I start?", "Is python still be relevant in 2025"]:
    conv = Conversation()
    conv.add_message(HumanMessage(content=prompt))
    conversations.append(conv)

In [20]:
batch_results = await model.abatch(conversations=conversations)
for i, result in enumerate(batch_results, 1):
    print(f"Answer to question {i} is:")
    print(result.get_last().content)
    print()  

Answer to question 1 is:
Here are some steps to get started learning Python:

1. Install Python:
   - Go to the official Python website (https://www.python.org/) and download the latest version of Python for your operating system.
   - Follow the installation instructions for your platform.

2. Learn the basics:
   - Start with learning the basic syntax and language constructs of Python, such as variables, data types, operators, control structures (if-else, loops), and functions.
   - There are many free online resources, such as tutorials, documentation, and interactive coding platforms, that can help you learn the basics.

3. Practice, practice, practice:
   - The best way to learn Python is by writing and executing code. Start with simple programs and gradually work your way up to more complex projects.
   - You can find a variety of beginner-friendly coding exercises and projects online to practice your skills.

4. Explore Python libraries and modules:
   - Python has a vast ecosys

### **Conclusion:**  
In this notebook, we were able to cover the asynchronous methods provided in the Swarmauri LLM classes. You will truly appreciate the beauty of the **async** methods when building real-world applications that require speed and need to handle a lot of tasks at the same time.  

As you can see, the async methods are just like the sync methods, and that is one of the beauties of using Swarmauri. We make it very easy to use as all models have the same methods and the same way of interacting with them.  

In this notebook, we used **Anthropic** as an example, but you can use any model of your choice, and the code will work perfectly as long as you import the correct class and instantiate it. In the previous example, we used **Groq**, but you could also use any other model there, and it would work just as well. Swarmauri ensures that the methods are straightforward and consistent, regardless of the model you choose.  

Watch out for the next notebooks, where we will explore more exciting features of Swarmauri.

## **NOTEBOOK METADATA**

In [1]:
from swarmauri.utils import print_notebook_metadata

metadata = print_notebook_metadata.print_notebook_metadata("Victory Nnaji", "3rd-Son")
print(metadata) 

Author: Victory Nnaji
GitHub Username: 3rd-Son
Notebook File: Notebook_03_Async_Methods_with_AnthropicModel.ipynb
Last Modified: 2024-12-30 07:38:51.784408
Platform: Darwin 24.1.0
Python Version: 3.11.11 (main, Dec 11 2024, 10:25:04) [Clang 14.0.6 ]
Swarmauri Version: 0.5.2
None
